# (Optional) Colab Setup
If you aren't using Colab, you can delete the following code cell. This is just to help students with mounting to Google Drive to access the other .py files and downloading the data, which is a little trickier on Colab than on your local machine using Jupyter. 

In [4]:
# you will be prompted with a window asking to grant permissions
from google.colab import drive
drive.mount("/content/drive")

ModuleNotFoundError: No module named 'google'

In [ ]:
# fill in the path in your Google Drive in the string below. Note: do not escape slashes or spaces
import os
datadir = "/content/assignment1"
if not os.path.exists(datadir):
  !ln -s "/content/drive/My Drive/YOUR PATH HERE/assignment1/" $datadir
os.chdir(datadir)
!pwd

In [ ]:
# downloading Fashion-MNIST
import os
os.chdir(os.path.join(datadir,"fashion-mnist/"))
!chmod +x ./get_data.sh
!./get_data.sh
os.chdir(datadir)

# Imports

In [2]:
import random
import numpy as np
from data_process import get_FASHION_data, get_RICE_data
from scipy.spatial import distance
from models import Perceptron, SVM, Softmax, Logistic
from kaggle_submission import output_submission_csv
%matplotlib inline

# For auto-reloading external modules
# See http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Loading Fashion-MNIST

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 60000]
, TEST_IMAGES = 10000

In [3]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 50000
VAL_IMAGES = 10000
normalize = True

In [4]:
data = get_FASHION_data(TRAIN_IMAGES, VAL_IMAGES, normalize=normalize)
X_train_fashion, y_train_fashion = data['X_train'], data['y_train']
X_val_fashion, y_val_fashion = data['X_val'], data['y_val']
X_test_fashion, y_test_fashion = data['X_test'], data['y_test']
n_class_fashion = len(np.unique(y_test_fashion))

# Loading Rice

In [5]:
# loads train / test / val splits of 80%, 20%, 20% 
data = get_RICE_data()
X_train_RICE, y_train_RICE = data['X_train'], data['y_train']
X_val_RICE, y_val_RICE = data['X_val'], data['y_val']
X_test_RICE, y_test_RICE = data['X_test'], data['y_test']
n_class_RICE = len(np.unique(y_test_RICE))

print("Number of train samples: ", X_train_RICE.shape[0])
print("Number of val samples: ", X_val_RICE.shape[0])
print("Number of test samples: ", X_test_RICE.shape[0])

Number of train samples:  10911
Number of val samples:  3637
Number of test samples:  3637


### Get Accuracy

This function computes how well your model performs using accuracy as a metric.

In [6]:
def get_acc(pred, y_test):
    return np.sum(y_test == pred) / len(y_test) * 100

# Perceptron

Perceptron has 2 hyperparameters that you can experiment with:
### Learning rate
The learning rate controls how much we change the current weights of the classifier during each update. We set it at a default value of 0.5, but you should experiment with different values. Here is a guide to help you find a right learning rate: 
- Try values ranging from 5.0 to 0.0005 to see the impact on model accuracy. 
- If the accuracy fluctuates a lot or diverges, the learning rate is too high. Try decreasing it by a factor of 10 (e.g. from 0.5 to 0.05). 
- If the accuracy is changing very slowly, the learning rate may be too low. Try increasing it by a factor of 10.
- You can also try adding a learning rate decay to slowly reduce the learning rate over each training epoch. For example, multiply the learning rate by 0.95 after each epoch.
- Plot training and validation accuracy over epochs for different learning rates. This will help you visualize the impact of the learning rate.
- [Here](https://towardsdatascience.com/https-medium-com-dashingaditya-rakhecha-understanding-learning-rate-dd5da26bb6de) is a detailed guide to learning rate.

### Number of Epochs
An epoch is a complete iterative pass over all of the data in the dataset. During an epoch we predict a label using the classifier and then update the weights of the classifier according to the perceptron update rule for each sample in the training set. You should try different values for the number of training epochs and report your results.

You will implement the Perceptron classifier in the **models/perceptron.py**

The following code: 
- Creates an instance of the Perceptron classifier class 
- The train function of the Perceptron class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy


## Train Perceptron on Fashion-MNIST

In [217]:
lr = 0.05
n_epochs = 20

percept_fashion = Perceptron(n_class_fashion, lr, n_epochs)
percept_fashion.train(X_train_fashion, y_train_fashion)

Epoch 1/20, Accuracy: 75.30%
Epoch 2/20, Accuracy: 77.27%
Epoch 3/20, Accuracy: 78.96%
Epoch 4/20, Accuracy: 78.82%
Epoch 5/20, Accuracy: 79.74%
Epoch 6/20, Accuracy: 78.53%
Epoch 7/20, Accuracy: 80.20%
Epoch 8/20, Accuracy: 79.12%
Epoch 9/20, Accuracy: 79.79%
Epoch 10/20, Accuracy: 78.93%
Epoch 11/20, Accuracy: 80.14%
Epoch 12/20, Accuracy: 77.68%
Epoch 13/20, Accuracy: 77.92%
Epoch 14/20, Accuracy: 78.64%
Epoch 15/20, Accuracy: 79.22%
Epoch 16/20, Accuracy: 79.33%
Epoch 17/20, Accuracy: 78.35%
Epoch 18/20, Accuracy: 80.47%
Epoch 19/20, Accuracy: 79.15%
Epoch 20/20, Accuracy: 80.62%


In [218]:
pred_percept = percept_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_fashion)))

The training accuracy is given by: 80.620000


### Validate Perceptron on Fashion-MNIST

In [219]:
pred_percept = percept_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_fashion)))

The validation accuracy is given by: 78.390000


### Test Perceptron on Fashion-MNIST

In [220]:
pred_percept = percept_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_fashion)))

The testing accuracy is given by: 77.430000


### Perceptron_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy, output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [81]:
output_submission_csv('kaggle/perceptron_submission_fashion.csv', percept_fashion.predict(X_test_fashion))

## Train Perceptron on Rice

In [223]:
lr = 0.05
n_epochs = 50

percept_RICE = Perceptron(n_class_RICE, lr, n_epochs)
percept_RICE.train(X_train_RICE, y_train_RICE)

Epoch 1/50, Accuracy: 69.44%
Epoch 2/50, Accuracy: 66.58%
Epoch 3/50, Accuracy: 72.05%
Epoch 4/50, Accuracy: 83.71%
Epoch 5/50, Accuracy: 66.68%
Epoch 6/50, Accuracy: 94.39%
Epoch 7/50, Accuracy: 97.42%
Epoch 8/50, Accuracy: 96.64%
Epoch 9/50, Accuracy: 97.50%
Epoch 10/50, Accuracy: 98.76%
Epoch 11/50, Accuracy: 98.15%
Epoch 12/50, Accuracy: 97.00%
Epoch 13/50, Accuracy: 99.18%
Epoch 14/50, Accuracy: 98.49%
Epoch 15/50, Accuracy: 98.14%
Epoch 16/50, Accuracy: 97.10%
Epoch 17/50, Accuracy: 99.92%
Epoch 18/50, Accuracy: 97.67%
Epoch 19/50, Accuracy: 97.73%
Epoch 20/50, Accuracy: 97.62%
Epoch 21/50, Accuracy: 97.70%
Epoch 22/50, Accuracy: 97.16%
Epoch 23/50, Accuracy: 98.22%
Epoch 24/50, Accuracy: 98.26%
Epoch 25/50, Accuracy: 97.38%
Epoch 26/50, Accuracy: 98.96%
Epoch 27/50, Accuracy: 99.29%
Epoch 28/50, Accuracy: 98.41%
Epoch 29/50, Accuracy: 98.45%
Epoch 30/50, Accuracy: 99.16%
Epoch 31/50, Accuracy: 99.38%
Epoch 32/50, Accuracy: 98.67%
Epoch 33/50, Accuracy: 99.76%
Epoch 34/50, Accura

In [224]:
pred_percept = percept_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_RICE)))

The training accuracy is given by: 99.505087


### Validate Perceptron on Rice

In [225]:
pred_percept = percept_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_RICE)))

The validation accuracy is given by: 99.532582


### Test Perceptron on Rice

In [226]:
pred_percept = percept_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_RICE)))

The testing accuracy is given by: 99.505087


# Support Vector Machines (with SGD)

Next, you will implement a "soft margin" SVM. In this formulation you will maximize the margin between positive and negative training examples and penalize margin violations using a hinge loss.

We will optimize the SVM loss using SGD. This means you must compute the loss function with respect to model weights. You will use this gradient to update the model weights.

SVM optimized with SGD has 3 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Epochs** - similar to as defined above in Perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case it is a coefficient on the term which maximizes the margin. You could try different values. The default value is set to 0.05.

You will implement the SVM using SGD in the **models/svm.py**

The following code: 
- Creates an instance of the SVM classifier class 
- The train function of the SVM class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train SVM on Fashion-MNIST

In [127]:
lr = [0.5, 0.05, 0.005]
n_epochs = [10,20,30]
reg_const =[0.05, 0.01]

best_lr = 0
best_epoch = 0
best_reg = 0
max_accu =0

for l in lr:
  for epoch in n_epochs:
    for reg in reg_const:
      print(f"Training with learning rate {l}, epoch {epoch}, reg_const{reg}...")
      svm_fashion = SVM(n_class_fashion, l, epoch, reg)
      accu = svm_fashion.train(X_train_fashion, y_train_fashion)
      if (accu > max_accu):
        best_lr = l
        best_epoch = epoch
        best_reg = reg
print(f"Best learning rate:{best_lr}, best epoch number:{best_epoch}, best reg const:{best_reg}")

# It turns out that they aren't the best hyper-parameters due to overfit

Training with learning rate 0.5, epoch 10, reg_const0.05...
Start training...
Epoch 1/10, Accuracy: 81.50%
Epoch 2/10, Accuracy: 77.41%
Epoch 3/10, Accuracy: 80.82%
Epoch 4/10, Accuracy: 82.71%
Epoch 5/10, Accuracy: 80.98%
Epoch 6/10, Accuracy: 83.02%
Epoch 7/10, Accuracy: 82.47%
Epoch 8/10, Accuracy: 82.81%
Epoch 9/10, Accuracy: 82.99%
Epoch 10/10, Accuracy: 84.23%
Training with learning rate 0.5, epoch 10, reg_const0.01...
Start training...
Epoch 1/10, Accuracy: 81.94%
Epoch 2/10, Accuracy: 80.05%
Epoch 3/10, Accuracy: 81.27%
Epoch 4/10, Accuracy: 82.35%
Epoch 5/10, Accuracy: 81.42%
Epoch 6/10, Accuracy: 82.50%
Epoch 7/10, Accuracy: 83.43%
Epoch 8/10, Accuracy: 82.15%
Epoch 9/10, Accuracy: 84.09%
Epoch 10/10, Accuracy: 82.10%
Training with learning rate 0.5, epoch 20, reg_const0.05...
Start training...
Epoch 1/20, Accuracy: 81.50%
Epoch 2/20, Accuracy: 77.41%
Epoch 3/20, Accuracy: 80.82%
Epoch 4/20, Accuracy: 82.71%
Epoch 5/20, Accuracy: 80.98%
Epoch 6/20, Accuracy: 83.02%
Epoch 7/20

In [233]:
pred_svm = svm_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_fashion)))

The training accuracy is given by: 84.226000


### Validate SVM on Fashion-MNIST

In [234]:
pred_svm = svm_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_fashion)))

The validation accuracy is given by: 83.150000


### Test SVM on Fashion-MNIST

In [235]:
pred_svm = svm_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_fashion)))

The testing accuracy is given by: 82.260000


### SVM_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [136]:
output_submission_csv('kaggle/svm_submission_fashion.csv', svm_fashion.predict(X_test_fashion))

## Train SVM on Rice

In [7]:
lr = 0.05
n_epochs = 100
reg_const = 0.05

svm_RICE = SVM(n_class_RICE, lr, n_epochs, reg_const)
svm_RICE.train(X_train_RICE, y_train_RICE)

Start training...
Epoch 1/100, Accuracy: 77.57%
Epoch 2/100, Accuracy: 71.94%
Epoch 3/100, Accuracy: 71.20%
Epoch 4/100, Accuracy: 58.87%
Epoch 5/100, Accuracy: 75.09%
Epoch 6/100, Accuracy: 73.68%
Epoch 7/100, Accuracy: 64.93%
Epoch 8/100, Accuracy: 68.89%
Epoch 9/100, Accuracy: 80.15%
Epoch 10/100, Accuracy: 80.07%
Epoch 11/100, Accuracy: 79.11%
Epoch 12/100, Accuracy: 81.34%
Epoch 13/100, Accuracy: 75.08%
Epoch 14/100, Accuracy: 79.95%
Epoch 15/100, Accuracy: 80.55%
Epoch 16/100, Accuracy: 81.87%
Epoch 17/100, Accuracy: 78.76%
Epoch 18/100, Accuracy: 82.80%
Epoch 19/100, Accuracy: 56.53%
Epoch 20/100, Accuracy: 81.96%
Epoch 21/100, Accuracy: 70.59%
Epoch 22/100, Accuracy: 81.10%
Epoch 23/100, Accuracy: 54.66%
Epoch 24/100, Accuracy: 67.36%
Epoch 25/100, Accuracy: 80.52%
Epoch 26/100, Accuracy: 83.70%
Epoch 27/100, Accuracy: 78.87%
Epoch 28/100, Accuracy: 84.32%
Epoch 29/100, Accuracy: 75.91%
Epoch 30/100, Accuracy: 78.29%
Epoch 31/100, Accuracy: 87.08%
Epoch 32/100, Accuracy: 84.30%

82.77600586564016

In [8]:
pred_svm = svm_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_RICE)))

The training accuracy is given by: 93.126203


### Validate SVM on Rice

In [9]:
pred_svm = svm_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_RICE)))

The validation accuracy is given by: 93.813583


## Test SVM on Rice

In [10]:
pred_svm = svm_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_RICE)))

The testing accuracy is given by: 92.823756


# Softmax Classifier (with SGD)

Next, you will train a Softmax classifier. This classifier consists of a linear function of the input data followed by a softmax function which outputs a vector of dimension C (number of classes) for each data point. Each entry of the softmax output vector corresponds to a confidence in one of the C classes, and like a probability distribution, the entries of the output vector sum to 1. We use a cross-entropy loss on this sotmax output to train the model. 

Check the following link as an additional resource on softmax classification: http://cs231n.github.io/linear-classify/#softmax

Once again we will train the classifier with SGD. This means you need to compute the gradients of the softmax cross-entropy loss function according to the weights and update the weights using this gradient. Check the following link to help with implementing the gradient updates: https://deepnotes.io/softmax-crossentropy

The softmax classifier has 3 hyperparameters that you can experiment with:
- **Learning rate** - As above, this controls how much the model weights are updated with respect to their gradient.
- **Number of Epochs** - As described for perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case, we minimize the L2 norm of the model weights as regularization, so the regularization constant is a coefficient on the L2 norm in the combined cross-entropy and regularization objective.

You will implement a softmax classifier using SGD in the **models/softmax.py**

The following code: 
- Creates an instance of the Softmax classifier class 
- The train function of the Softmax class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train Softmax on Fashion-MNIST

In [179]:
lr = 0.05
n_epochs = 20
reg_const = 0.5

softmax_fashion = Softmax(n_class_fashion, lr, n_epochs, reg_const)
softmax_fashion.train(X_train_fashion, y_train_fashion)

Start training...
Epoch 1/20, Accuracy: 82.04%
Epoch 2/20, Accuracy: 80.49%
Epoch 3/20, Accuracy: 81.51%
Epoch 4/20, Accuracy: 77.77%
Epoch 5/20, Accuracy: 79.66%
Epoch 6/20, Accuracy: 84.04%
Epoch 7/20, Accuracy: 77.72%
Epoch 8/20, Accuracy: 81.02%
Epoch 9/20, Accuracy: 83.59%
Epoch 10/20, Accuracy: 83.33%
Epoch 11/20, Accuracy: 82.45%
Epoch 12/20, Accuracy: 78.03%
Epoch 13/20, Accuracy: 81.65%
Epoch 14/20, Accuracy: 77.89%
Epoch 15/20, Accuracy: 80.75%
Epoch 16/20, Accuracy: 82.36%
Epoch 17/20, Accuracy: 83.59%
Epoch 18/20, Accuracy: 82.49%
Epoch 19/20, Accuracy: 84.32%
Epoch 20/20, Accuracy: 83.87%


81.42830000000001

In [180]:
pred_softmax = softmax_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_fashion)))

The training accuracy is given by: 83.866000


### Validate Softmax on Fashion-MNIST

In [181]:
pred_softmax = softmax_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_fashion)))

The validation accuracy is given by: 81.490000


### Testing Softmax on Fashion-MNIST

In [182]:
pred_softmax = softmax_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_fashion)))

# lr=0.5, epoch=10, reg=0.5: 79
# lr=0.05, epoch=20, reg=0.5: 80.53
# lr = 0.05, epoch=30, reg=0.3: 79.06
# lr = 0.05, epoch=30 reg=0.5: 79.06

The testing accuracy is given by: 80.530000


### Softmax_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [183]:
output_submission_csv('kaggle/softmax_submission_fashion.csv', softmax_fashion.predict(X_test_fashion))

## Train Softmax on Rice

In [11]:
lr = 0.05
n_epochs = 100
reg_const = 0.05

softmax_RICE = Softmax(n_class_RICE, lr, n_epochs, reg_const)
softmax_RICE.train(X_train_RICE, y_train_RICE)

Start training...
Epoch 1/100, Accuracy: 74.51%
Epoch 2/100, Accuracy: 77.33%
Epoch 3/100, Accuracy: 78.26%
Epoch 4/100, Accuracy: 77.98%
Epoch 5/100, Accuracy: 78.38%
Epoch 6/100, Accuracy: 68.47%
Epoch 7/100, Accuracy: 79.87%
Epoch 8/100, Accuracy: 80.74%
Epoch 9/100, Accuracy: 71.71%
Epoch 10/100, Accuracy: 76.14%
Epoch 11/100, Accuracy: 79.74%
Epoch 12/100, Accuracy: 77.79%
Epoch 13/100, Accuracy: 81.60%
Epoch 14/100, Accuracy: 78.76%
Epoch 15/100, Accuracy: 70.72%
Epoch 16/100, Accuracy: 81.87%
Epoch 17/100, Accuracy: 83.20%
Epoch 18/100, Accuracy: 80.39%
Epoch 19/100, Accuracy: 77.29%
Epoch 20/100, Accuracy: 80.31%
Epoch 21/100, Accuracy: 69.92%
Epoch 22/100, Accuracy: 79.61%
Epoch 23/100, Accuracy: 84.58%
Epoch 24/100, Accuracy: 77.18%
Epoch 25/100, Accuracy: 84.62%
Epoch 26/100, Accuracy: 80.92%
Epoch 27/100, Accuracy: 84.34%
Epoch 28/100, Accuracy: 65.21%
Epoch 29/100, Accuracy: 86.02%
Epoch 30/100, Accuracy: 83.65%
Epoch 31/100, Accuracy: 84.21%
Epoch 32/100, Accuracy: 87.06%

85.75043534048211

In [12]:
pred_softmax = softmax_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_RICE)))

The training accuracy is given by: 99.230135


### Validate Softmax on Rice

In [13]:
pred_softmax = softmax_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_RICE)))

The validation accuracy is given by: 99.340115


### Testing Softmax on Rice

In [14]:
pred_softmax = softmax_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_RICE)))

The testing accuracy is given by: 99.202640


# Logistic Classifier

The Logistic Classifier has 2 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Number of Epochs** - As described for perceptron.
- **Threshold** - The decision boundary of the classifier.


You will implement the Logistic Classifier in the **models/logistic.py**

The following code: 
- Creates an instance of the Logistic classifier class 
- The train function of the Logistic class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

### Training Logistic Classifer

In [37]:
learning_rate = [0.5, 0.05, 0.005]
n_epochs = [10,20,30]
threshold = 0.5

best_lr = 0
best_epoch = 0
best_thre = 0
max_accu =0

for l in learning_rate:
  for epoch in n_epochs:
      print(f"Training with learning rate {l}, epoch {epoch}, threshold 0.5 ...")
      lr= Logistic(l, epoch, threshold)
      accu = lr.train(X_train_RICE, y_train_RICE)
      if (accu > max_accu):
        best_lr = l
        best_epoch = epoch
        accu = 0
print(f"Best learning rate:{best_lr}, best epoch number:{best_epoch}, best threshold:{best_thre}")


lr= Logistic(best_lr, best_epoch, threshold)
lr.train(X_train_RICE, y_train_RICE)


Training with learning rate 0.5, epoch 10, threshold 0.5 ...
Start training...
Epoch 1/10, Accuracy: 69.36%
Epoch 2/10, Accuracy: 69.81%
Epoch 3/10, Accuracy: 68.40%
Epoch 4/10, Accuracy: 92.32%
Epoch 5/10, Accuracy: 94.03%
Epoch 6/10, Accuracy: 93.93%
Epoch 7/10, Accuracy: 96.25%
Epoch 8/10, Accuracy: 97.22%
Epoch 9/10, Accuracy: 99.39%
Epoch 10/10, Accuracy: 99.22%
Training with learning rate 0.5, epoch 20, threshold 0.5 ...
Start training...
Epoch 1/20, Accuracy: 69.36%
Epoch 2/20, Accuracy: 69.81%
Epoch 3/20, Accuracy: 68.40%
Epoch 4/20, Accuracy: 92.32%
Epoch 5/20, Accuracy: 94.03%
Epoch 6/20, Accuracy: 93.93%
Epoch 7/20, Accuracy: 96.25%
Epoch 8/20, Accuracy: 97.22%
Epoch 9/20, Accuracy: 99.39%
Epoch 10/20, Accuracy: 99.22%
Epoch 11/20, Accuracy: 99.33%
Epoch 12/20, Accuracy: 99.22%
Epoch 13/20, Accuracy: 97.35%
Epoch 14/20, Accuracy: 98.12%
Epoch 15/20, Accuracy: 99.78%
Epoch 16/20, Accuracy: 97.63%
Epoch 17/20, Accuracy: 99.37%
Epoch 18/20, Accuracy: 98.73%
Epoch 19/20, Accurac

94.9579934622552

In [38]:
pred_lr = lr.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_lr, y_train_RICE)))

The training accuracy is given by: 99.578407


### Validate Logistic Classifer

In [39]:
pred_lr = lr.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_RICE)))

The validation accuracy is given by: 99.560077


### Test Logistic Classifier

In [40]:
pred_lr = lr.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_RICE)))

The testing accuracy is given by: 99.532582
